In [1]:
import requests
import pandas as pd

In [2]:
ruta_archivo = r"C:\Users\caste\DataspellProjects\cf-ml\ProyectoSECOP2\datos_filtradosv1.csv"
# Cargamos los datos del CSV a un DataFrame de pandas
df = pd.read_csv(ruta_archivo)

C:\Users\caste\AppData\Local\Temp\ipykernel_35464\3374636744.py:3: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta_archivo)


In [6]:
# Seleccionamos las columnas que queremos mantener
df = df[['Nombre Entidad', 'Nit Entidad', 'Departamento', 'Ciudad',
         'Localización', 'Orden', 'Sector', 'Rama', 'Entidad Centralizada',
         'Estado Contrato',
         'Tipo de Contrato',
         'Modalidad de Contratacion',
         'Fecha de Firma', 'Fecha de Inicio del Contrato',
         'Fecha de Fin del Contrato', 'Fecha de Inicio de Ejecucion',
         'Fecha de Fin de Ejecucion', 'Condiciones de Entrega',
         'TipoDocProveedor',
         'Es Grupo', 'Es Pyme', 'Habilita Pago Adelantado', 'Liquidación',
         'Obligación Ambiental', 'Obligaciones Postconsumo', 'Reversion',
         'Valor del Contrato', 'Valor de pago adelantado', 'Valor Facturado',
         'Valor Pendiente de Pago', 'Valor Pagado', 'Valor Amortizado',
         'Valor Pendiente de Amortizacion', 'Valor Pendiente de Ejecucion',
         'Saldo CDP',
         'Saldo Vigencia', 'EsPostConflicto', 'Destino Gasto',
         'Origen de los Recursos', 'Dias Adicionados',
         'Nacionalidad Representante Legal',
         'Género Representante Legal',
         'Presupuesto General de la Nacion – PGN',
         'Sistema General de Participaciones', 'Sistema General de Regalías',
         'Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)',
         'Recursos de Credito', 'Recursos Propios',
         'Fecha Inicio Liquidacion', 'Fecha Fin Liquidacion'
         ]]

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2689335 entries, 0 to 2689334
Data columns (total 50 columns):
 #   Column                                                              Dtype  
---  ------                                                              -----  
 0   Nombre Entidad                                                      object 
 1   Nit Entidad                                                         int64  
 2   Departamento                                                        object 
 3   Ciudad                                                              object 
 4   Localización                                                        object 
 5   Orden                                                               object 
 6   Sector                                                              object 
 7   Rama                                                                object 
 8   Entidad Centralizada                                                obje

#### Filtrar por el estado del contrato

#### Conversion de las fechas a DateTime

In [9]:
columnas_fecha = ['Fecha de Firma', 'Fecha de Inicio del Contrato', 'Fecha de Fin del Contrato',
                  'Fecha de Inicio de Ejecucion', 'Fecha de Fin de Ejecucion', 'Fecha Inicio Liquidacion',
                  'Fecha Fin Liquidacion']

# Iterar sobre las columnas y convertirlas al tipo de dato 'date'
for columna in columnas_fecha:
    df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
    df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')

C:\Users\caste\AppData\Local\Temp\ipykernel_35464\239712058.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = pd.to_datetime(df[columna], format='%m/%d/%Y', errors='coerce')
C:\Users\caste\AppData\Local\Temp\ipykernel_35464\239712058.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columna] = pd.to_datetime(df[columna], format='%Y-%m-%d %H:%M:%S', errors='coerce')
C:\Users\caste\AppData\Local\Temp\ipykernel_35464\239712058.py:7: SettingWithCopyWarning: 
A value is trying to be set on a

In [10]:
estados_contrato = ["terminado", "Cerrado", "cedido", "Prorrogado", "Suspendido"]

# Filtrar las filas del dataframe basándote en la columna "Estado Contrato"
df_filtrado = df[df["Estado Contrato"].isin(estados_contrato)]
df_filtrado.shape

(608240, 50)

#### Filtrar por año del contrato

In [12]:
df_filtradoAnio = df_filtrado[df_filtrado['Fecha de Inicio del Contrato'].dt.year >= 2017]
df_filtradoAnio

,Nombre Entidad,Nit Entidad,Departamento,Ciudad,Localización,Orden,Sector,Rama,Entidad Centralizada,Estado Contrato,...,Nacionalidad Representante Legal,Género Representante Legal,Presupuesto General de la Nacion – PGN,Sistema General de Participaciones,Sistema General de Regalías,"Recursos Propios (Alcaldías, Gobernaciones y Resguardos Indígenas)",Recursos de Credito,Recursos Propios,Fecha Inicio Liquidacion,Fecha Fin Liquidacion
0,ALCALDIA DE FLORIDABLANCA,890205176,Santander,Floridablanca,"Colombia, Santander, Floridablanca",Territorial,Servicio Público,Ejecutivo,Centralizada,terminado,...,COLOMBIANA,Hombre,0,0,0,13750000,0,0,NaT,NaT
2,SENA REGIONAL ATLANTICO,899999034,Atlántico,Barranquilla,"Colombia, Atlántico , Barranquilla",Nacional,Educación Nacional,Ejecutivo,Centralizada,cedido,...,Colombiana,No Definido,0,0,0,0,0,0,NaT,NaT
3,MINISTERIO DE HACIENDA Y CREDITO PUBLICO,899999090,Distrito Capital de Bogotá,No Definido,"Colombia, Bogotá, Bogotá",Nacional,Hacienda y Crédito Público,Ejecutivo,No Definido,Cerrado,...,colombiano,No Definido,0,0,0,0,0,0,NaT,NaT
9,AGENCIA DE DESARROLLO RURAL,900948958,Distrito Capital de Bogotá,No Definido,"Colombia, Bogotá, Bogotá",Nacional,agricultura,Ejecutivo,Descentralizada,terminado,...,Colombiana,Mujer,22032000,0,0,0,0,0,NaT,NaT
10,INSTITUTO COLOMBIANO PARA LA EVALUACIÓN DE LA ...,860024301,Distrito Capital de Bogotá,Bogotá,"Colombia, Bogotá, Bogotá",Nacional,Educación Nacional,Corporación Autónoma,Descentralizada,terminado,...,Colombiano,No Definido,0,0,0,0,0,49500000,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2689323,ASAMBLEA DEPARTAMENTAL DE CUNDINAMARCA,899999716,Cundinamarca,No Definido,"Colombia, Cundinamarca",Territorial,No aplica/No pertenece,Ejecutivo,Descentralizada,Cerrado,...,Colombiana,No Definido,0,0,0,0,0,0,NaT,NaT
2689327,POLICIA METROPOLITANA DE NEIVA,900634185,Huila,Neiva,"Colombia, Huila, Neiva",Nacional,defensa,Ejecutivo,Descentralizada,terminado,...,COLOMBIANA,No Definido,24500000,0,0,0,0,0,NaT,NaT
2689331,SUBRED INTEGRADA DE SERVICIOS DE SALUD NORTE ESE,900971006,Distrito Capital de Bogotá,No Definido,"Colombia, Bogotá, Bogotá",Territorial,Salud y Protección Social,Corporación Autónoma,Descentralizada,terminado,...,Colombiana,Mujer,0,0,0,0,0,6957491,NaT,NaT
2689332,SENAREGIONAL ATLANTICO CENTRO PARA EL DESARROL...,899999034,Atlántico,Sabanalarga,"Colombia, Atlántico , Sabanalarga",Nacional,Trabajo,Ejecutivo,Centralizada,cedido,...,colombiana,No Definido,20680000,0,0,0,0,0,NaT,NaT
